## TP 4 : " Reaction - Diffusion Model ".

---
Binôme : Khaled Ibrahim & Servranckx Christophe.

---


Pour ce 4ème travail personnel, nous allons nous intéresser au modèle Gray-scott et à son système d'équations.


Le modèle Gray-Scott, comme nous l'avons vu dans l'introduction de ce TP, représente la réaction et la diffusion de deux espèces chimique générique que l'on notera U et V. 


Dès lors, l'objectif sera ici, sous certaine condition initiales (CI) et au bords (CB), de simuler l'interaction des deux espèces chimique qui réagissent entre-elle et se diffusent.


Pour se faire, nous aurons à discrétiser le système d'équation différentiel partielle (EDP) selon une discrétisation *forward* pour le temps et *centrale* pour l'espace et à implémenter notre système à deux dimensions comme nous l'avons fait tout au long de ce 4ème module.

---

##### 1. Rappel et développement théorique.
---

et dont les concentrations, en un point de l'espace, sont représentés par les variables u et v respectivement.

---

##### 2. Discrétisation des équations de réaction - diffusion et des conditions au bord.
---


\begin{align}
\frac{\partial u}{\partial t} &= D_u \nabla ^2 u - uv^2 + F(1-u)\\
\frac{\partial v}{\partial t} &= D_v \nabla ^2 v + uv^2 - (F + k)v
\end{align}

Comme indiqué, nous discrétisons les équations de réaction-diffusion en utilisant les discrétisations *forward-time* et *centered-space*. Procédons tout d'abord à la discrétisation de l'équation (1).


---

#### 2.1 Discrétisation de l'équation (1)
---

Commençons par la discrétisation des dérivées temporelle et spatiale.

* Forward-time : 

$$ \frac{\partial u}{\partial t} \approx \frac{u^{n+1}_{i,j} - u^{n}_{i,j}}{\Delta t} $$

* centered-space : 

$$ \nabla ^2 u \approx \frac{u^{n}_{i+1,j} - 2u^{n}_{i,j} + u^{n}_{i-1,j}}{\Delta x^2} + \frac{u^{n}_{i,j+1} - 2u^{n}_{i,j} + u^{n}_{i,j-1}}{\Delta y^2} $$

$\hspace{0.8cm}$ ou en supposant que $ \Delta x = \Delta y = \delta$. 

$$→  \nabla ^2 u \approx \frac{1}{\delta ^2} \left[ \left( u^{n}_{i+1,j} - 2u^{n}_{i,j} + u^{n}_{i-1,j} \right) + \left( u^{n}_{i,j+1} - 2u^{n}_{i,j} + u^{n}_{i,j-1} \right) \right]$$

Notre équation discrétisée est ainsi donnée par :

$$\frac{u^{n+1}_{i,j} - u^{n}_{i,j}}{\Delta t} = \frac{D_u}{\delta ^2} \left[ \left( u^{n}_{i+1,j} - 2u^{n}_{i,j} + u^{n}_{i-1,j} \right) + \left( u^{n}_{i,j+1} - 2u^{n}_{i,j} + u^{n}_{i,j-1} \right) \right] - u^{n}_{i,j} \left( v^n_{i,j} \right) ^2 + F \left(1-u^{n}_{i,j} \right) $$

Où la seule inconnue est $u^{n+1}_{i,j}$. Dès lors, en isolant cette inconnue dans le membre de gauche:

$$
\begin{array}{}
u^{n+1}_{i,j}= u^{n}_{i,j} + \frac{D_u \Delta t}{\delta ^2} \left[ \left( u^{n}_{i+1,j} - 2u^{n}_{i,j} + u^{n}_{i-1,j} \right) + \left( u^{n}_{i,j+1} - 2u^{n}_{i,j} + u^{n}_{i,j-1} \right) \right] - \Delta{t} \mbox{ } u^{n}_{i,j} \left( v^n_{i,j} \right) ^2 + \Delta{t} \mbox{ } F \left(1-u^{n}_{i,j} \right)
\end{array}
$$

et en réarrangant quelque peu cette équation, nous obtenons :

$$
\begin{array}{}
\hspace{-3.0cm} u^{n+1}_i = \frac{D_u \Delta t}{\delta ^2} \left( u^{n}_{i+1,j} + u^{n}_{i-1,j} + u^{n}_{i,j+1} + u^{n}_{i,j-1} \right) + \left[ 1- \frac{4 D_u \Delta t}{\delta ^2} - \Delta{t} \mbox{ } \left( v^n_{i,j} \right) ^2 - \Delta{t} \mbox{ } F \right] u^{n}_{i,j} + \Delta{t} \mbox{ } F 
\end{array}
$$
 
Finalement, en posant $c_1 = \frac{D_u \Delta t}{\delta ^2}$, il en découle l'équation suivante:

$$
\begin{equation}
\hspace{-2.0cm} u^{n+1}_i = c_1 \left( u^{n}_{i+1,j} + u^{n}_{i-1,j} + u^{n}_{i,j+1} + u^{n}_{i,j-1} \right) \hspace{1.0cm} + \left[ 1- 4 c_1 - \Delta{t} \mbox{ } \left( v^n_{i,j} \right) ^2 - \Delta{t} \mbox{ } F \right] u^{n}_{i,j} + \Delta{t} \mbox{ } F 
\end{equation}
$$

---

#### 2.2 Discrétisation de l'équation (2)
---

Nous procédons de manière identique à ce que nous venons d'éffectuer pour l'équation (1), e.g., nous commençons par discrétiser les dérivées temporelle et spatiale,

* Forward-time : 

$$ \frac{\partial v}{\partial t} \approx \frac{v^{n+1}_{i,j} - v^{n}_{i,j}}{\Delta t} $$

* centered-space : 

$$ \nabla ^2 v \approx \frac{v^{n}_{i+1,j} - 2v^{n}_{i,j} + v^{n}_{i-1,j}}{\Delta x^2} + \frac{v^{n}_{i,j+1} - 2v^{n}_{i,j} + v^{n}_{i,j-1}}{\Delta y^2} $$

$\hspace{0.8cm}$ Or, $ \Delta x = \Delta y = \delta$.

$$→  \nabla ^2 v \approx \frac{1}{\delta ^2} \left[ \left( v^{n}_{i+1,j} - 2v^{n}_{i,j} + v^{n}_{i-1,j} \right) + \left( v^{n}_{i,j+1} - 2v^{n}_{i,j} + v^{n}_{i,j-1} \right) \right]$$

qui nous donne,

$$ \frac{v^{n+1}_{i,j} - v^{n}_{i,j}}{\Delta t} = \frac{D_v}{\delta ^2} \left[ \left( v^{n}_{i+1,j} - 2v^{n}_{i,j} + v^{n}_{i-1,j} \right) + \left( v^{n}_{i,j+1} - 2v^{n}_{i,j} + v^{n}_{i,j-1} \right) \right] - u^{n}_{i,j} \left( v^{n}_{i,j} \right) ^2 - \left(F+k \right) v^{n}_{i,j} $$

En isolant la seule inconnue $v^{n+1}_{i,j}$, nous avons alors:

$$ 
\begin{array}{}
v^{n+1}_{i,j} = v^{n}_{i,j} + \frac{D_v \Delta t}{\delta ^2} \left[ \left( v^{n}_{i+1,j} - 2v^{n}_{i,j} + v^{n}_{i-1,j} \right) + \left( v^{n}_{i,j+1} - 2v^{n}_{i,j} + v^{n}_{i,j-1} \right) \right] + \Delta{t} \mbox{ } u^{n}_{i,j} \left( v^{n}_{i,j} \right) ^2 - \Delta{t} \left(F+k \right) v^{n}_{i,j} 
\end{array}
$$

qui devient après un réarrangement et en posant $c_2 = \frac{D_v \Delta t}{\delta ^2}$ :

$$
\begin{equation}
\hspace{-1.0cm} v^{n+1}_{i,j} = c_2 \left( v^{n}_{i+1,j} + v^{n}_{i-1,j} + v^{n}_{i,j+1} + v^{n}_{i,j-1} \right) + \left[ 1- 4 c_2 - \Delta{t} \mbox{ } \left( F+k \right)  \right] v^{n}_{i,j} + \Delta{t} \mbox{ } u^{n}_{i,j} \left( v^{n}_{i,j} \right) ^2
\end{equation}
$$

Nous disposons à présent des équations de réaction-diffusion 

$$
\left\{
\begin{array}{}
u^{n+1}_i = c_1 \left( u^{n}_{i+1,j} + u^{n}_{i-1,j} + u^{n}_{i,j+1} + u^{n}_{i,j-1} \right) \hspace{1.0cm} + \left[ 1- 4 c_1 - \Delta{t} \mbox{ } \left( v^n_{i,j} \right) ^2 - \Delta{t} \mbox{ } F \right] u^{n}_{i,j} + \Delta{t} \mbox{ } F  \\
v^{n+1}_{i,j} = c_2 \left( v^{n}_{i+1,j} + v^{n}_{i-1,j} + v^{n}_{i,j+1} + v^{n}_{i,j-1} \right) + \left[ 1- 4 c_2 - \Delta{t} \mbox{ } \left( F+k \right)  \right] v^{n}_{i,j} + \Delta{t} \mbox{ } u^{n}_{i,j} \left( v^{n}_{i,j} \right) ^2
\end{array}
\right.
$$

Il nous reste alors qu'a discrétisé nos conditions aux bords.

---

##### 2.3 Conditions aux bords.
---


Pour les 4 bords du domaines nous disposons d'une même CB à savoir, une CB de Neumann nulle. Or comme nous l'avons vu au fil des leçons de ce module, nous  pouvons approximer les CB de Neumann par des différences finies. Nous avons donc pour ce problème :

$$
\left\{
\begin{array}{}
\frac{\partial u}{\partial x} = \frac{\partial u}{\partial y} = q\\
\frac{\partial v}{\partial x} = \frac{\partial v}{\partial y} = q
\end{array}
\right.
$$

et où $q=0$ (ce qui correspond à une paroie qui isole notre système de son environnement). Ainsi, à chaque pas de temps nous avons (en se souvenant que $ \Delta x = \Delta y = \delta$):

* Pour u :

$$
→ \left\{
\begin{array}{}
\frac{\partial u}{\partial x} \approx \frac{u_{end,j} - u_{end-1,j}}{\Delta x} = q \mbox{ } → \mbox{ } u_{end,j} = u_{end-1,j} + q \delta\\
\frac{\partial u}{\partial x} \approx \frac{u_{i,end} - u_{i,end-1}}{\Delta y} = q \mbox{ } → \mbox{ } u_{i,end} = u_{i,end-1} + q \delta 
\end{array}
\right.
$$

* Pour v :

$$
→ \left\{
\begin{array}{}
\frac{\partial v}{\partial x} \approx \frac{v_{end,j} - v_{end-1,j}}{\Delta x} = q \mbox{ } → \mbox{ } v_{end,j} = v_{end-1,j} + q \delta\\
\frac{\partial v}{\partial x} \approx \frac{v_{i,end} - v_{i,end-1}}{\Delta y} = q \mbox{ } → \mbox{ } v_{i,end} = v_{i,end-1} + q \delta 
\end{array}
\right.
$$

---

##### 3.
---

In [1]:
import numpy
from matplotlib import pyplot
import matplotlib.cm as cm
%matplotlib inline

In [3]:
n = 192
Du, Dv, F, k = 0.00016, 0.00008, 0.035, 0.065 # Bacteria 1 
dh = 5/(n-1)
T = 8000
dt = .9 * dh**2 / (4*max(Du,Dv))
nt = int(T/dt)

In [4]:
uvinitial = numpy.load('./data/uvinitial.npz')
U = uvinitial['U']
V = uvinitial['V']

---
###### La cellule ci-dessous charge le style du notebook.

In [1]:
from IPython.core.display import HTML
css_file = '../styles/numericalmoocstyle.css'
HTML(open(css_file, "r").read())